### Import libraries

In [2]:
import pandas as pd 
import numpy as np
import os 
import requests
import json
from datetime import datetime

In [3]:
# Create a list with all files in the folder using a list compehension

folderpath = r"/Users/prateeksharma/Downloads/2022-citibike-tripdata" 
filepaths = [os.path.join(folderpath, name) for name in os.listdir(folderpath) if name.endswith('.csv')]

In [6]:
# Initialize an empty list to hold dataframes
dataframes = []

# Define dtype for columns with mixed types
dtype_dict = {
    'ride_id': str,
    'rideable_type': str,
    'started_at': str,
    'ended_at': str,
    'start_station_name': str,
    'start_station_id': str, 
    'end_station_name': str,
    'end_station_id': str, 
}

# Loop through the list of file paths and read each CSV file
for file in filepaths:
    try:
        # Attempt to read the file with UTF-8 encoding
        df = pd.read_csv(file, encoding='utf-8', dtype=dtype_dict, low_memory=False)
    except UnicodeDecodeError:
        # If UTF-8 encoding fails, try ISO-8859-1 encoding
        df = pd.read_csv(file, encoding='ISO-8859-1', dtype=dtype_dict, low_memory=False)
    except Exception as e:
        # Handle any other exceptions that might occur
        print(f"Error reading {file}: {e}")
        continue

    # Convert 'start_station_id' and 'end_station_id' to numeric, coercing errors to NaN
    df['start_station_id'] = pd.to_numeric(df['start_station_id'], errors='coerce')
    df['end_station_id'] = pd.to_numeric(df['end_station_id'], errors='coerce')

    # Append the dataframe to the list
    dataframes.append(df)

This code initializes an empty list dataframes to store pandas DataFrames. It defines dtype_dict to specify that certain columns should be read as strings, which helps handle mixed data types. The code then iterates over a list of file paths (filepaths) and attempts to read each CSV file. It first tries to read each file with UTF-8 encoding. If a UnicodeDecodeError occurs, it retries with ISO-8859-1 encoding. If any other exception occurs during file reading, it catches the error, prints an error message, and skips to the next file. Successfully read DataFrames are appended to the dataframes list.

In [9]:
df = pd.concat(dataframes, ignore_index=True)

In [11]:
df.shape

(30689921, 13)

In [13]:
df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,DF208007EE5F9D79,classic_bike,2022-08-26 15:21:44,2022-08-26 15:34:20,Berkeley Pl & 6 Ave,4134.06,Windsor Pl & 8 Ave,3620.02,40.676530,-73.978469,40.660906,-73.983074,member
1,48A796B4996BACB0,electric_bike,2022-08-03 12:56:29,2022-08-03 13:06:10,21 St & 4 Ave,3668.04,Degraw St & Smith St,4298.05,40.662584,-73.995554,40.682915,-73.993182,member
2,AE459F435C0B1D5B,classic_bike,2022-08-11 17:49:47,2022-08-11 18:05:08,48 St & Barnett Ave,6358.05,Queens Plaza North & Crescent St,6429.01,40.750016,-73.915409,40.751102,-73.940737,member
3,CA8B4A94620E74C0,electric_bike,2022-08-10 17:25:41,2022-08-10 17:35:23,7 Ave & Central Park South,6912.01,8 Ave & W 33 St,6450.12,40.766741,-73.979069,40.751551,-73.993934,member
4,EB85C6F0148BC348,classic_bike,2022-08-31 13:47:12,2022-08-31 14:24:00,Jerome Ave & W 195 St,8653.09,Sherman Ave & E 166 St,8072.06,40.868858,-73.896500,40.831305,-73.917448,member


In [15]:
df.tail()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
30689916,CB2EE50F60A58C7B,classic_bike,2022-12-12 08:38:46,2022-12-12 08:46:12,E 41 St & Madison Ave (SW corner),6474.12,6 Ave & W 33 St,6364.07,40.752165,-73.979922,40.749013,-73.988484,member
30689917,3C0DF3280FEB1350,classic_bike,2022-12-12 13:06:32,2022-12-12 13:19:58,Peck Slip & South St,5096.12,Vesey St & Church St,5216.06,40.707689,-74.001254,40.712220,-74.010472,member
30689918,B916E63B630D5020,classic_bike,2022-12-18 01:15:22,2022-12-18 01:48:15,E 41 St & Madison Ave (SW corner),6474.12,Lenox Ave & W 111 St,7602.05,40.752399,-73.980150,40.798786,-73.952300,member
30689919,21EAC007BE7D4161,classic_bike,2022-12-08 23:09:54,2022-12-08 23:26:37,W 37 St & 10 Ave,6611.02,W 70 St & Amsterdam Ave,7207.08,40.756604,-73.997901,40.777480,-73.982886,member
30689920,538408C4A16F4415,electric_bike,2022-12-21 07:26:14,2022-12-21 07:31:40,W 37 St & 10 Ave,6611.02,6 Ave & W 33 St,6364.07,40.756604,-73.997901,40.749013,-73.988484,member


### Get weather data using NOAA's API

In [18]:
# Define your NOAA token - type yours in here! 

Token = 'BiEOLsQXtdVgcqzWoCpANtIcqqEMcOlZ' 

In [20]:
# Get the API 

r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=TAVG&limit=1000&stationid=GHCND:USW00014732&startdate=2022-01-01&enddate=2022-12-31', headers={'token':Token})

In [22]:
# Load the api response as a json

d = json.loads(r.text)

In [24]:
# Secure all items in the response that correspond to TAVG

avg_temps = [item for item in d['results'] if item['datatype']=='TAVG']

In [26]:
# Get only the date field from all average temperature readings

dates_temp = [item['date'] for item in avg_temps]

In [28]:
# Get the temperature from all average temperature readings

temps = [item['value'] for item in avg_temps]

In [30]:
first_10_temps = temps[:10]
print(first_10_temps)

[116, 114, 14, -27, 32, 49, 7, -25, 14, 16]


In [32]:
# Put the results in a dataframe

df_temp = pd.DataFrame()

In [34]:
# Get only date and cast it to date time; convert temperature from tenths of Celsius to normal Celsius
df_temp['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
df_temp['avgTemp'] = [float(v) / 10.0 for v in temps]

In [36]:
df_temp.tail()

,date,avgTemp
360,2022-12-27,-0.7
361,2022-12-28,3.4
362,2022-12-29,6.4
363,2022-12-30,9.3
364,2022-12-31,8.2


In [38]:
df_temp.head()

,date,avgTemp
0,2022-01-01,11.6
1,2022-01-02,11.4
2,2022-01-03,1.4
3,2022-01-04,-2.7
4,2022-01-05,3.2


In [40]:
df['started_at'] = pd.to_datetime(df['started_at'], format="%Y-%m-%d %H:%M:%S", errors='coerce')

In [41]:
df['date'] = pd.to_datetime(df['started_at'], format='%Y-%m-%d').dt.date

In [42]:
df['date'] = pd.to_datetime(df['date'])

In [43]:
df_temp.head()

,date,avgTemp
0,2022-01-01,11.6
1,2022-01-02,11.4
2,2022-01-03,1.4
3,2022-01-04,-2.7
4,2022-01-05,3.2


In [48]:
df_temp.to_csv('NY_LaGuardia_data.csv')

In [50]:
%%time
df_merged = df.merge(df_temp, how = 'left', on = 'date', indicator = True)

CPU times: user 13.5 s, sys: 18.1 s, total: 31.7 s
Wall time: 38.3 s


In [52]:
df_merged.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,date,avgTemp,_merge
0,DF208007EE5F9D79,classic_bike,2022-08-26 15:21:44,2022-08-26 15:34:20,Berkeley Pl & 6 Ave,4134.06,Windsor Pl & 8 Ave,3620.02,40.676530,-73.978469,40.660906,-73.983074,member,2022-08-26,27.4,both
1,48A796B4996BACB0,electric_bike,2022-08-03 12:56:29,2022-08-03 13:06:10,21 St & 4 Ave,3668.04,Degraw St & Smith St,4298.05,40.662584,-73.995554,40.682915,-73.993182,member,2022-08-03,27.6,both
2,AE459F435C0B1D5B,classic_bike,2022-08-11 17:49:47,2022-08-11 18:05:08,48 St & Barnett Ave,6358.05,Queens Plaza North & Crescent St,6429.01,40.750016,-73.915409,40.751102,-73.940737,member,2022-08-11,26.6,both
3,CA8B4A94620E74C0,electric_bike,2022-08-10 17:25:41,2022-08-10 17:35:23,7 Ave & Central Park South,6912.01,8 Ave & W 33 St,6450.12,40.766741,-73.979069,40.751551,-73.993934,member,2022-08-10,27.2,both
4,EB85C6F0148BC348,classic_bike,2022-08-31 13:47:12,2022-08-31 14:24:00,Jerome Ave & W 195 St,8653.09,Sherman Ave & E 166 St,8072.06,40.868858,-73.896500,40.831305,-73.917448,member,2022-08-31,25.6,both


In [54]:
df_merged['_merge'].value_counts(dropna = False)

both          30689921
left_only            0
right_only           0
Name: _merge, dtype: int64

In [56]:
df_merged.to_csv('NY_CitiBike_data.csv')